In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchaudio
from os.path import join as pjoin
import matplotlib.pyplot as plt
import os
import tqdm
import time
from IPython.display import Audio #listen: ipd.Audio(real.detach().cpu().numpy(), rate=FS)
import numpy as np
import scipy
import scipy.signal as sig
import pandas as pd
import torchmetrics.audio as M
from speechmos import dnsmos
from datetime import datetime

def rep_list(short, long):
    reps = int(np.ceil(len(long) / len(short)))
    short *= reps
    short = short[:len(long)]
    return short
    
def plot_tensor(x):
    plt.plot(x.cpu().detach().numpy())

def extend_signal(signal, target_length):
    """
    Extend a signal by repeating it if it's shorter than the target length.
    
    Args:
    signal (torch.Tensor): Input signal.
    target_length (int): Desired length of the extended signal.

    Returns:
    torch.Tensor: Extended signal.
    """
    current_length = signal.size(0)
    if current_length < target_length:
        repetitions = target_length // current_length
        remainder = target_length % current_length
        extended_signal = signal.repeat(repetitions)
        if remainder > 0:
            extended_signal = torch.cat((extended_signal, signal[:remainder]), dim=0)
        return extended_signal
    else:
        return signal

def load_audio(apath):
    audio, fs = torchaudio.load(apath)
    if fs != FS:
        #print('resampling')
        resampler = torchaudio.transforms.Resample(fs, FS)
        audio = resampler(audio)    
    if len(audio.shape) > 1:
            audio = audio[0,:]
    return audio

def power(signal):
    return np.mean(signal**2)

/home/ubuntu/enric/venvs/dns/lib/python3.8/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [2]:
FS = 48000
DURATION = 4 #time in seconds of the eval chunk
TRAINRIR_NAMES = {'D01_sb_none_NH_mono': 'singleband' , 'D02_mb_none_NH_mono': 'multiband', 
            'D03_mb_rec_NH_left': 'recdirectivity', 'D05_mb_srcrec_NH_left': 'recsourcedirectivity',
            'D00_DNS5': 'DNS5'}

use_gpu = True
if torch.cuda.is_available() and use_gpu:
    TORCH_DEVICE = "cuda"
else:
    TORCH_DEVICE = "cpu"

batch_size = 1
num_workers = 8
reverberant_noises = True
speech_path = '/home/ubuntu/Data/DFN/textfiles/test_set.txt'
noise_path = '/home/ubuntu/Data/DFN/textfiles/test_set_noise.txt'
dns_mos_path = '/home/ubuntu/enric/DNS-Challenge/DNSMOS/DNSMOS'


In [50]:
air_path = '/home/ubuntu/Data/AIR_1_4'
air_rirs = os.listdir(air_path)

In [51]:
bin_rirs = []
for x in air_rirs:
    if 'binaural' in x:
        bin_rirs.append(x)

In [52]:
bin_rir_paths.sort()

In [60]:
left_bin_rirs = []
for bin_rir_path in bin_rir_paths:
    if scipy.io.loadmat(pjoin(air_path, bin_rir_path))['air_info'][0][0][5] == 1:
        left_bin_rirs.append(scipy.io.loadmat(pjoin(air_path, bin_rirs[1]))['h_air'][0])
        

In [61]:
left_bin_rirs = np.array(left_bin_rirs)

In [62]:
left_bin_rirs.shape

(100, 479376)

In [68]:
for i, rir in enumerate(left_bin_rirs):
    torchaudio.save(uri=pjoin(pjoin(air_path, 'left_bin_rirs'), 'leftBrir_'+str(i)+'.wav'), src=torch.from_numpy(rir).unsqueeze(0), sample_rate=48000)

In [75]:
left_bin_rirs = [pjoin(pjoin(air_path, 'left_bin_rirs'), x) for x in  os.listdir(pjoin(air_path, 'left_bin_rirs'))]

In [76]:
with open('AIR_brirs_left.txt', 'w') as file:
    # Write each item on a new line
    for item in left_bin_rirs:
        file.write(f"{item}\n")